# Kuzushiji Recognition: Plan

Objectives:
- Establish working GPU environment and baseline quickly; iterate to medal.

High-level plan:
1) Environment
   - Verify GPU (nvidia-smi) and install CUDA 12.1 PyTorch stack.
   - Keep constraints to avoid accidental torch upgrades.

2) Data and EDA
   - Inspect train.csv schema, class distribution, and file availability.
   - Unzip images to fast local storage; verify counts and a few samples.

3) Validation
   - Stratified K-Fold (by target) to mirror test distribution.
   - Save folds to disk; consistent reuse.

4) Baseline model
   - Image classifier (timm, e.g., resnet50/efficientnetv2_s) with mixed precision.
   - 224→384 resolution sweep; augmentations: flips, rotate, slight affine, contrast.
   - BCE/CE depending on label format; compute F1 on OOF.

5) Iterate
   - Improve preprocessing (grayscale, CLAHE, thresholding) and augmentations.
   - Try larger backbones and test-time augmentation.
   - Ensemble diverse seeds/backbones if time.

6) Submission
   - Predict test set, build submission.csv in required format, sanity-check head and schema.

Checkpoints to request expert review:
- After this plan and environment setup.
- After EDA/validation design.
- After first baseline OOF.
- After any major architecture/resolution change and before long runs.

Time discipline:
- Log progress per fold/epoch; early stop.
- Start with small smoke runs (1-2 epochs, 1-2 folds) before full training.

In [1]:
# Environment & Quick EDA
import os, sys, subprocess, time, re, zipfile, json
import pandas as pd

def run(cmd):
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout

print('===== nvidia-smi =====', flush=True)
print(run(['bash','-lc','nvidia-smi || true']))

print('===== Torch check =====', flush=True)
try:
    import torch
    print('torch:', torch.__version__, 'cuda build:', getattr(torch.version, 'cuda', None), 'CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('GPU:', torch.cuda.get_device_name(0))
except Exception as e:
    print('Torch not available or import failed:', e)

print('===== List files =====')
for f in sorted(os.listdir('.')):
    print(f, os.path.getsize(f))

print('\n===== CSV heads =====')
train_path = 'train.csv'
ss_path = 'sample_submission.csv'
unicode_path = 'unicode_translation.csv'
df = pd.read_csv(train_path)
print('train.csv shape:', df.shape)
print(df.head(3))
print('\nColumns:', df.columns.tolist())
ss = pd.read_csv(ss_path)
print('\nsample_submission head:')
print(ss.head(3))
unicode_df = pd.read_csv(unicode_path)
print('\nunicode_translation head:')
print(unicode_df.head(3))

label_col = None
for c in df.columns:
    if c.lower() in ('labels','label','annotations','targets'):
        label_col = c
        break
if label_col is None:
    # try last column if two columns image_id + labels pattern
    if df.shape[1] >= 2:
        label_col = df.columns[-1]
print('Guessed label column:', label_col)

def classify_task(sample_labels):
    # If any row contains 'U+' and numeric coords, it's detection-style string
    det_pat = re.compile(r'U\+[0-9A-Fa-f]{4,6}')
    coord_pat = re.compile(r'\b\d+\b')
    for s in sample_labels:
        s = '' if pd.isna(s) else str(s)
        if det_pat.search(s):
            nums = coord_pat.findall(s)
            if len(nums) >= 4:
                return 'detection'
    return 'classification'

task_type = None
if label_col is not None:
    sample_vals = df[label_col].astype(str).head(5).tolist()
    task_type = classify_task(sample_vals)
print('Inferred task type:', task_type)

if label_col is not None and task_type == 'detection':
    # Quick stats: proportion empty, count codepoints
    det_pat = re.compile(r'(U\+[0-9A-Fa-f]{4,6})')
    empties = (df[label_col].fillna('').str.strip() == '').mean()
    print('Empty label rows fraction:', round(empties, 4))
    all_codes = []
    for s in df[label_col].fillna(''):
        all_codes += det_pat.findall(s)
    vc = pd.Series(all_codes).value_counts().head(10)
    print('Top codepoints:', vc.to_dict())

print('\n===== Peek inside image zips =====')
for zpath in ['train_images.zip','test_images.zip']:
    try:
        with zipfile.ZipFile(zpath) as zf:
            names = [n for n in zf.namelist() if n.lower().endswith(('.jpg','.jpeg','.png'))]
            print(zpath, 'num images:', len(names))
            print('First 5:', names[:5])
    except Exception as e:
        print('Error reading', zpath, e)

print('\nDONE EDA SNAPSHOT', flush=True)

===== nvidia-smi =====


Sun Sep 28 18:37:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Torch not available or import failed: No module named 'torch'
===== List files =====
.00_eda_and_planning_kernel_state.json 182
00_eda_and_planning.ipynb 6901
agent_metadata 4096
description.md 10914
docker_run.log 45164
requirements.txt 2021
sample_submission.csv 13700
submission.csv 13700
task.txt 1003
test_images.zip 307454375
train.csv 14069467
train_images.zip 2711943152
unicode_translation.csv 52646

===== CSV heads =====
train.csv shape: (3244, 2)
            image_id                                             labels
0  200004148_00015_1  U+306F 1187 361 47 27 U+306F 1487 2581 48 28 U...
1  200021712-00008_2  U+4E00 1543 1987 58 11 U+4E00 1296 1068 91 11 ...
2  100249416_00034_1  U+4E00 1214 415 73 11 U+4E00 1386 412 72 13 U+...

Columns: ['image_id', 'labels']

sample_submission head:
            image_id                 labels
0        umgy007-028  U+003F 1 1 U+FF2F 2 2
1        hnsd004-026  U+003F 1 1 U+FF2F 2 2
2  200003076_00034_2  U+003F 1 1 U+FF2F 2 2

unicode_translatio

Top codepoints: {'U+306B': 22169, 'U+306E': 21680, 'U+3057': 19910, 'U+3066': 18443, 'U+3068': 14900, 'U+3092': 14233, 'U+306F': 13250, 'U+304B': 12878, 'U+308A': 12692, 'U+306A': 12369}

===== Peek inside image zips =====
train_images.zip num images: 3244
First 5: ['200021637-00009_1.jpg', '200015779_00037_2.jpg', 'umgy006-022.jpg', '200015779_00110_1.jpg', 'brsk002-024.jpg']
test_images.zip num images: 361
First 5: ['umgy010-043.jpg', 'hnsd005-026.jpg', '200021660-00080_2.jpg', '200014740-00027_2.jpg', '200003076_00107_1.jpg']

DONE EDA SNAPSHOT


In [2]:
# Environment setup: install CUDA 12.1 torch stack and key libs
import os, sys, subprocess, shutil, time
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any stray torch stacks (idempotent)
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean possible shadow dirs
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Freeze constraints
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Install non-torch deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'ultralytics==8.3.5',
    'timm==1.0.9',
    'albumentations==1.4.14',
    'opencv-python-headless==4.10.0.84',
    'pandas', 'numpy', 'pyyaml', 'scikit-learn',
    'ensemble-boxes==1.0.9',
    '--upgrade-strategy', 'only-if-needed')

# Sanity check torch + CUDA
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))
print('Env setup complete.', flush=True)

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 155.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 163.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 168.2 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 77.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 353.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 94.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 503.3 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Env setup complete.


In [4]:
# Prepare YOLOv8 class-agnostic dataset (extract, parse labels, split, hardlink)
import os, re, zipfile, math, json, shutil, random, time
from pathlib import Path
from collections import defaultdict
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

# Use writable project-local directory instead of /mnt
base = Path('kuzu_data')
pool_dir = base / 'pool' / 'train'
img_train_dir = base / 'images' / 'train'
img_val_dir = base / 'images' / 'val'
lbl_all_dir = base / 'labels_all'
lbl_train_dir = base / 'labels' / 'train'
lbl_val_dir = base / 'labels' / 'val'
for p in [pool_dir, img_train_dir, img_val_dir, lbl_all_dir, lbl_train_dir, lbl_val_dir]:
    p.mkdir(parents=True, exist_ok=True)

def extract_zip_images(zip_path, out_dir):
    t0 = time.time()
    with zipfile.ZipFile(zip_path) as zf:
        names = [n for n in zf.namelist() if n.lower().endswith(('.jpg','.jpeg','.png'))]
        done = 0
        for n in names:
            dst = out_dir / Path(n).name
            if not dst.exists():
                with zf.open(n) as src, open(dst, 'wb') as f:
                    shutil.copyfileobj(src, f)
            done += 1
            if done % 200 == 0:
                print(f'Extracted {done}/{len(names)} in {time.time()-t0:.1f}s', flush=True)
    print(f'Extraction complete: {len(names)} files -> {out_dir} in {time.time()-t0:.1f}s')

# 1) Extract training images to pool
if len(list(pool_dir.glob('*.jpg'))) < 3000:
    extract_zip_images('train_images.zip', pool_dir)
else:
    print('Pool already extracted; skipping')

# 2) Read train.csv and parse labels into YOLO txt (class 0), normalized by image size
df = pd.read_csv('train.csv')
assert set(df.columns) >= {'image_id','labels'}
det_pat = re.compile(r'(U\+[0-9A-Fa-f]{4,6})')
num_pat = re.compile(r'\d+')

def parse_label_string(s):
    # returns list of (code, x, y, w, h) in pixel ints
    if pd.isna(s) or not str(s).strip():
        return []
    tokens = str(s).strip().split()
    out = []
    i = 0
    while i < len(tokens):
        tok = tokens[i]
        if tok.startswith('U+') and i + 5 <= len(tokens):
            code = tok
            try:
                x = int(tokens[i+1]); y = int(tokens[i+2]); w = int(tokens[i+3]); h = int(tokens[i+4])
                out.append((code, x, y, w, h))
            except Exception:
                pass
            i += 5
        else:
            i += 1
    return out

sizes_cache = {}
def get_size(img_name):
    if img_name in sizes_cache:
        return sizes_cache[img_name]
    p = pool_dir / img_name
    with Image.open(p) as im:
        w, h = im.size
    sizes_cache[img_name] = (w, h)
    return w, h

num_boxes_per_image = []
t0 = time.time()
for idx, row in df.iterrows():
    img_id = row['image_id']
    # image files appear to be .jpg
    img_name = f'{img_id}.jpg'
    img_path = pool_dir / img_name
    if not img_path.exists():
        # fallback: some ids contain '-' vs '_' variants already include .jpg in zip; try to find matching file
        candidates = list(pool_dir.glob(f'{img_id}.*'))
        if len(candidates) == 1:
            img_path = candidates[0]
            img_name = img_path.name
        else:
            # as a last resort skip
            print('Missing image for', img_id, '- skipping')
            continue
    W, H = get_size(img_name)
    anns = parse_label_string(row['labels'])
    num_boxes_per_image.append((img_name, len(anns)))
    # write label file
    lbl_path = lbl_all_dir / (Path(img_name).stem + '.txt')
    if not lbl_path.exists():
        with open(lbl_path, 'w') as f:
            for (_code, x, y, w, h) in anns:
                # YOLO expects cx, cy, w, h normalized
                cx = (x + w / 2.0) / W
                cy = (y + h / 2.0) / H
                f.write(f'0 {cx:.6f} {cy:.6f} {w / W:.6f} {h / H:.6f}\n')
    if (idx+1) % 200 == 0:
        print(f'Parsed {idx+1}/{len(df)} images in {time.time()-t0:.1f}s')
print('Label parsing complete in', f'{time.time()-t0:.1f}s')

# 3) Train/Val split stratified by number of boxes (binned)
nb = pd.DataFrame(num_boxes_per_image, columns=['img_name','n'])
bins = pd.cut(nb['n'], bins=[-1,5,15,30,60,120,10000], labels=False)
train_names, val_names = train_test_split(nb['img_name'].values, test_size=0.1, random_state=42, stratify=bins)
train_set = set(train_names); val_set = set(val_names)
print('Split sizes:', len(train_set), len(val_set))

# 4) Hardlink images and labels into split folders
def safe_link(src, dst):
    if dst.exists():
        return
    dst.parent.mkdir(parents=True, exist_ok=True)
    try:
        os.link(src, dst)
    except Exception:
        shutil.copy2(src, dst)

for phase, img_dir, lbl_dir, names in [('train', img_train_dir, lbl_train_dir, train_set), ('val', img_val_dir, lbl_val_dir, val_set)]:
    t0 = time.time()
    done = 0
    for name in names:
        src_img = pool_dir / name
        dst_img = img_dir / name
        safe_link(src_img, dst_img)
        src_lbl = lbl_all_dir / (Path(name).stem + '.txt')
        dst_lbl = lbl_dir / (Path(name).stem + '.txt')
        # some images might have zero boxes; ensure an empty file exists
        if not src_lbl.exists():
            open(src_lbl, 'a').close()
        safe_link(src_lbl, dst_lbl)
        done += 1
        if done % 300 == 0:
            print(f'{phase}: linked {done} in {time.time()-t0:.1f}s', flush=True)
    print(f'{phase}: linking complete: {done} files in {time.time()-t0:.1f}s')

# 5) Write data.yaml for Ultralytics
data_yaml = base / 'kuzu.yaml'
data_yaml.write_text('\n'.join([
    f'path: {base.resolve()}',
    'train: images/train',
    'val: images/val',
    'names: [glyph]',
    'nc: 1',
]))
print('Wrote', data_yaml)
print('Dataset prep complete.')

Extracted 200/3244 in 0.5s


Extracted 400/3244 in 1.0s


Extracted 600/3244 in 1.5s


Extracted 800/3244 in 2.0s


Extracted 1000/3244 in 2.5s


...

train: linked 2100 in 0.2s


train: linked 2400 in 0.2s


train: linked 2700 in 0.2s


train: linking complete: 2919 files in 0.2s
val: linked 300 in 0.0s


val: linking complete: 325 files in 0.0s
Wrote kuzu_data/kuzu.yaml
Dataset prep complete.


In [8]:
# Train YOLOv8s class-agnostic detector @1024 per expert config
import time
from ultralytics import YOLO

t0 = time.time()
model = YOLO('yolov8s.pt')
print('Starting YOLOv8s training...', flush=True)
results = model.train(
    data='kuzu_data/kuzu.yaml',
    imgsz=1024,
    batch=16,
    epochs=70,
    device=0,
    amp=True,
    cos_lr=True,
    patience=15,
    warmup_epochs=3,
    rect=True,
    cache='ram',
    workers=8,
    close_mosaic=10,
    degrees=7.0,
    fliplr=0.0,
    flipud=0.0,
    hsv_v=0.3,
    translate=0.1,
    scale=0.3,
    shear=2.0,
    mosaic=1.0,
    mixup=0.0,
    project='runs_det',
    name='yolov8s_1024_ca',
    pretrained=True,
    verbose=True,
    seed=42
)
print('Training finished in %.1f min' % ((time.time()-t0)/60.0), flush=True)
print('Best weights:', results.save_dir)

  0%|          | 0.00/21.5M [00:00<?, ?B/s]

 30%|███       | 6.50M/21.5M [00:00<00:00, 68.0MB/s]

 60%|██████    | 13.0M/21.5M [00:00<00:00, 53.3MB/s]

100%|█████████▉| 21.5M/21.5M [00:00<00:00, 67.2MB/s]

100%|██████████| 21.5M/21.5M [00:00<00:00, 64.6MB/s]

...

       9/70      19.8G     0.8204     0.4278     0.8658       2630        672:  54%|█████▎    | 98/183 [00:25<00:20,  4.23it/s]

       9/70      19.8G     0.8204     0.4278     0.8658       2630        672:  54%|█████▍    | 99/183 [00:25<00:20,  4.19it/s]

       9/70      19.8G     0.8205     0.4278     0.8657       2956        672:  54%|█████▍    | 99/183 [00:25<00:20,  4.19it/s]

       9/70      19.8G     0.8205     0.4278     0.8657       2956        672:  55%|█████▍    | 100/183 [00:25<00:19,  4.18it/s]

       9/70      19.8G     0.8202     0.4277     0.8657       3055        672:  55%|█████▍    | 100/183 [00:25<00:19,  4.18it/s]

In [6]:
# Fix OpenCV runtime deps (libGL) and sanity-check cv2 import
import subprocess, sys, os
def run(cmd):
    print('>',' '.join(cmd), flush=True)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(p.stdout, flush=True)
    return p.returncode

run(['bash','-lc','apt-get update -y && apt-get install -y libgl1 libglib2.0-0'])
try:
    import cv2
    print('cv2 version:', cv2.__version__)
except Exception as e:
    print('cv2 import failed:', e)

> bash -lc apt-get update -y && apt-get install -y libgl1 libglib2.0-0


Reading package lists...
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (30: Read-only file system)



cv2 import failed: libGL.so.1: cannot open shared object file: No such file or directory


In [7]:
# Ensure headless OpenCV is used (remove GUI OpenCV), then sanity-check imports
import sys, subprocess
def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=False)

# Uninstall any OpenCV packages that may require libGL
for pkg in ['opencv-python', 'opencv-contrib-python', 'opencv-python-headless']:
    pip('uninstall', '-y', pkg)

# Reinstall headless only
pip('install', 'opencv-python-headless==4.10.0.84')

# Test imports
try:
    import cv2
    print('cv2 OK, version:', cv2.__version__)
except Exception as e:
    print('cv2 import failed:', e)

try:
    from ultralytics import YOLO
    print('Ultralytics YOLO import OK')
except Exception as e:
    print('Ultralytics import failed:', e)

> pip uninstall -y opencv-python


Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
> pip uninstall -y opencv-contrib-python


> pip uninstall -y opencv-python-headless


Found existing installation: opencv-python-headless 4.10.0.84
Uninstalling opencv-python-headless-4.10.0.84:
  Successfully uninstalled opencv-python-headless-4.10.0.84
> pip install opencv-python-headless==4.10.0.84


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 218.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 206.8 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.5 requires opencv-python>=4.6.0, which is not installed.


cv2 OK, version: 4.10.0


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/app/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics YOLO import OK


In [11]:
# Local F1 evaluators (IoU>=0.5) and val inference/grid-search scaffold
import os, time, re, math, json
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from scipy.optimize import linear_sum_assignment

# --- GT parsing (from train.csv strings) ---
def parse_label_string(s):
    if pd.isna(s) or not str(s).strip():
        return []
    tokens = str(s).strip().split()
    out = []
    i = 0
    while i < len(tokens):
        tok = tokens[i]
        if tok.startswith('U+') and i + 5 <= len(tokens):
            try:
                code = tok
                x = int(tokens[i+1]); y = int(tokens[i+2]); w = int(tokens[i+3]); h = int(tokens[i+4])
                out.append((code, x, y, w, h))
            except Exception:
                pass
            i += 5
        else:
            i += 1
    return out

# --- Box helpers ---
def iou_xywh(a, b):
    # a,b: (x,y,w,h) in pixels
    ax1, ay1, aw, ah = a; ax2, ay2 = ax1 + aw, ay1 + ah
    bx1, by1, bw, bh = b; bx2, by2 = bx1 + bw, by1 + bh
    inter_w = max(0, min(ax2, bx2) - max(ax1, bx1))
    inter_h = max(0, min(ay2, by2) - max(ay1, by1))
    inter = inter_w * inter_h
    if inter <= 0: return 0.0
    ua = aw * ah + bw * bh - inter
    return inter / ua if ua > 0 else 0.0

def hungarian_match(gt_boxes, pr_boxes, iou_thresh=0.5, require_class=False):
    # gt_boxes: list of (code, x,y,w,h); pr_boxes: list of (code_or_None, x,y,w,h, conf)
    if len(gt_boxes) == 0 and len(pr_boxes) == 0:
        return 0, 0, 0, []  # TP, FP, FN, matches
    if len(gt_boxes) == 0:
        return 0, len(pr_boxes), 0, []
    if len(pr_boxes) == 0:
        return 0, 0, len(gt_boxes), []
    # Build cost matrix (1 - IoU), invalid (class-mismatch or IoU<iou_thresh) set to large cost
    G = len(gt_boxes); P = len(pr_boxes)
    cost = np.ones((G, P), dtype=np.float32) * 1.0
    for i, (gcode, gx, gy, gw, gh) in enumerate(gt_boxes):
        for j, (pcode, px, py, pw, ph, pc) in enumerate(pr_boxes):
            if require_class and (pcode is None or pcode != gcode):
                continue
            iou = iou_xywh((gx, gy, gw, gh), (px, py, pw, ph))
            if iou >= iou_thresh:
                cost[i, j] = 1.0 - iou
    gi, pj = linear_sum_assignment(cost)
    matched = []
    used_pred = set()
    tp = 0
    for a, b in zip(gi, pj):
        if cost[a, b] < 1.0:
            tp += 1
            used_pred.add(b)
            matched.append((a, b))
    fp = P - len(used_pred)
    fn = G - tp
    return tp, fp, fn, matched

def f1_from_counts(tp, fp, fn):
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    return (2 * prec * rec / (prec + rec)) if (prec + rec) > 0 else 0.0, prec, rec

# --- Load val GT ---
val_img_dir = Path('kuzu_data/images/val')
val_lbl_dir = Path('kuzu_data/labels/val')
train_csv = pd.read_csv('train.csv')
val_names = sorted([p.name for p in val_img_dir.glob('*.jpg')])
val_gt = {}  # img_name -> list of (code, x,y,w,h)
for name in val_names:
    img_id = Path(name).stem
    row = train_csv.loc[train_csv['image_id'] == img_id]
    if len(row) == 1:
        val_gt[name] = parse_label_string(row.iloc[0]['labels'])
    else:
        val_gt[name] = []
print('Val images:', len(val_names))

# --- Inference helper (Ultralytics) ---
def run_detector_on_val(weights_path, conf=0.2, iou=0.55, max_det=3000, agnostic_nms=True):
    from ultralytics import YOLO
    model = YOLO(weights_path)
    results = model.predict(source=str(val_img_dir), imgsz=1024, conf=conf, iou=iou,
                            agnostic_nms=agnostic_nms, max_det=max_det, verbose=False,
                            save=False, stream=True, device=0)
    preds = {}  # img_name -> list of (None, x,y,w,h, conf) since class-agnostic
    for r in results:
        p = Path(r.path).name
        preds[p] = []
        if r.boxes is not None and len(r.boxes) > 0:
            xyxy = r.boxes.xyxy.cpu().numpy()  # N x 4
            confs = r.boxes.conf.cpu().numpy()
            for (x1,y1,x2,y2), c in zip(xyxy, confs):
                x = int(round(x1)); y = int(round(y1));
                w = max(1, int(round(x2 - x1))); h = max(1, int(round(y2 - y1)));
                preds[p].append((None, x, y, w, h, float(c)))
    return preds

def eval_detector_only_f1(preds, iou_thresh=0.5):
    total_tp = total_fp = total_fn = 0
    for name, gt in val_gt.items():
        pr = preds.get(name, [])
        tp, fp, fn, _ = hungarian_match(gt, pr, iou_thresh=iou_thresh, require_class=False)
        total_tp += tp; total_fp += fp; total_fn += fn
    f1, prec, rec = f1_from_counts(total_tp, total_fp, total_fn)
    return f1, prec, rec, (total_tp, total_fp, total_fn)

# --- Grid search scaffold (will run AFTER training completes) ---
def wait_for_weights(run_dir='runs_det/yolov8s_1024_ca'):
    run = Path(run_dir)
    best = run / 'weights' / 'best.pt'
    last = run / 'weights' / 'last.pt'
    for _ in range(180):  # wait up to ~30 min in 10s steps
        if best.exists():
            return str(best)
        if last.exists():
            return str(last)
        time.sleep(10)
    return None

def grid_search_after_training():
    w = wait_for_weights()
    if w is None:
        print('Weights not ready yet.')
        return None
    print('Using weights:', w)
    best = {'f1': -1, 'conf': None, 'iou': None, 'prec': None, 'rec': None}
    for conf in np.linspace(0.10, 0.30, 5):
        for iou in np.linspace(0.40, 0.70, 7):
            preds = run_detector_on_val(w, conf=float(conf), iou=float(iou), max_det=3000, agnostic_nms=True)
            f1, prec, rec, cnts = eval_detector_only_f1(preds, iou_thresh=0.5)
            print(f'conf={conf:.2f} iou={iou:.2f} -> F1={f1:.4f} P={prec:.4f} R={rec:.4f} TP,FP,FN={cnts}')
            if f1 > best['f1']:
                best.update({'f1': f1, 'conf': float(conf), 'iou': float(iou), 'prec': prec, 'rec': rec})
    print('Best grid result:', best)
    return best

print('Evaluator and grid-search utilities ready. Call grid_search_after_training() once training finishes.')

Val images: 325
Evaluator and grid-search utilities ready. Call grid_search_after_training() once training finishes.


In [36]:
# Test images extract + detection scaffold (save raw boxes for later classification)
import os, time, zipfile, json
from pathlib import Path
import numpy as np

base = Path('kuzu_data')
test_pool = base / 'pool' / 'test'
test_img_dir = base / 'images' / 'test'
for p in [test_pool, test_img_dir]:
    p.mkdir(parents=True, exist_ok=True)

def extract_test_images(zip_path='test_images.zip', out_dir=test_pool):
    with zipfile.ZipFile(zip_path) as zf:
        names = [n for n in zf.namelist() if n.lower().endswith(('.jpg','.jpeg','.png'))]
        done = 0
        t0 = time.time()
        for n in names:
            dst = out_dir / Path(n).name
            if not dst.exists():
                with zf.open(n) as src, open(dst, 'wb') as f:
                    f.write(src.read())
            done += 1
            if done % 100 == 0:
                print(f'Extracted {done}/{len(names)} in {time.time()-t0:.1f}s', flush=True)
    print('Test extraction complete:', len(names))

def link_test_images(pool=test_pool, dest=test_img_dir):
    imgs = sorted([p for p in pool.glob('*.jpg')])
    for pth in imgs:
        dst = dest / pth.name
        if not dst.exists():
            try:
                os.link(pth, dst)
            except Exception:
                import shutil as _sh
                _sh.copy2(pth, dst)
    print('Linked test images:', len(imgs))

def detect_test(weights_path, out_path='det_test_raw.npy', conf=0.2, iou=0.55, max_det=3000):
    from ultralytics import YOLO
    model = YOLO(weights_path)
    results = model.predict(source=str(test_img_dir), imgsz=1024, conf=conf, iou=iou,
                            agnostic_nms=True, max_det=max_det, verbose=False, save=False, stream=True, device=0)
    out = {}  # image_name -> list of [x,y,w,h,conf]
    for r in results:
        name = Path(r.path).name
        boxes = []
        if r.boxes is not None and len(r.boxes) > 0:
            xyxy = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            for (x1,y1,x2,y2), c in zip(xyxy, confs):
                x = int(round(x1)); y = int(round(y1));
                w = max(1, int(round(x2 - x1))); h = max(1, int(round(y2 - y1)));
                boxes.append([x,y,w,h,float(c)])
        out[name] = boxes
    np.save(out_path, out, allow_pickle=True)
    print('Saved detections to', out_path, 'with', len(out), 'images')
    return out_path

print('Test scaffold ready: call extract_test_images(); link_test_images() after training and threshold tuning.')

Test scaffold ready: call extract_test_images(); link_test_images() after training and threshold tuning.


In [17]:
# Crop-classifier scaffolding: Unicode map, Dataset, and training loop (to run after detector tuning)
import os, json, math, time, random
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

base = Path('kuzu_data')
pool_train = base / 'pool' / 'train'
train_csv = pd.read_csv('train.csv')

# --- Build Unicode <-> id maps from train only (persist to disk) ---
def parse_label_string(s):
    if pd.isna(s) or not str(s).strip():
        return []
    tokens = str(s).strip().split()
    out = []
    i = 0
    while i < len(tokens):
        tok = tokens[i]
        if tok.startswith('U+') and i + 5 <= len(tokens):
            try:
                code = tok
                x = int(tokens[i+1]); y = int(tokens[i+2]); w = int(tokens[i+3]); h = int(tokens[i+4])
                out.append((code, x, y, w, h))
            except Exception:
                pass
            i += 5
        else:
            i += 1
    return out

codes = []
for _, r in train_csv.iterrows():
    for code, x, y, w, h in parse_label_string(r['labels']):
        codes.append(code)
uc2id = {c:i for i,c in enumerate(sorted(set(codes)))}
id2uc = {i:c for c,i in uc2id.items()}
(base / 'unicode_to_id.json').write_text(json.dumps(uc2id))
(base / 'id_to_unicode.json').write_text(json.dumps(id2uc))
num_classes = len(uc2id)
print('Unicode classes:', num_classes)

# --- Crop dataset using GT boxes (padding, grayscale->3ch). For training classifier only. ---
class KuzuCropTrainDataset(Dataset):
    def __init__(self, df, img_root, img_names=None, img_size=224, pad_pct=0.04, transforms=None):
        self.df = df
        self.img_root = Path(img_root)
        self.img_size = img_size
        self.pad_pct = pad_pct
        self.uc2id = uc2id
        self.items = []  # list of (img_path, x,y,w,h, class_id)
        if img_names is None:
            img_names = set(p.name for p in (base / 'images' / 'train').glob('*.jpg'))
        lookup = {Path(n).stem: n for n in img_names}
        for _, r in df.iterrows():
            image_id = r['image_id']
            if image_id not in lookup:  # skip images held-out for detector val, optional
                continue
            img_name = lookup[image_id]
            anns = parse_label_string(r['labels'])
            for code, x, y, w, h in anns:
                cls = self.uc2id.get(code, None)
                if cls is None: continue
                self.items.append((img_name, x, y, w, h, cls))
        self.transforms = transforms or A.Compose([
            A.Resize(self.img_size, self.img_size, interpolation=1),
            A.Rotate(limit=5, border_mode=0, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.CoarseDropout(max_holes=2, max_height=int(0.15*self.img_size), max_width=int(0.15*self.img_size), p=0.3),
            ToTensorV2(),
        ])
        print('Crop items:', len(self.items))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        img_name, x, y, w, h, cls = self.items[idx]
        img_path = self.img_root / img_name
        with Image.open(img_path) as im:
            im = im.convert('L')  # grayscale
            W, H = im.size
            # FIX: pad relative to box size, not page size
            pad = int(round(self.pad_pct * max(w, h)))
            x1 = max(0, x - pad); y1 = max(0, y - pad)
            x2 = min(W, x + w + pad); y2 = min(H, y + h + pad)
            crop = im.crop((x1, y1, x2, y2))
            # center-pad to square then resize via A
            cw, ch = crop.size
            s = max(cw, ch)
            bg = Image.new('L', (s, s), color=255)
            bg.paste(crop, ((s - cw)//2, (s - ch)//2))
            crop = bg
            arr = np.array(crop)
            arr = np.stack([arr, arr, arr], axis=-1)  # 3ch
        aug = self.transforms(image=arr)
        img = aug['image']  # CxHxW float tensor 0-1
        target = torch.tensor(cls, dtype=torch.long)
        return img, target

# --- Classifier training loop ---
def create_sampler(dataset):
    # class-balanced sampler
    cls_counts = np.zeros(num_classes, dtype=np.int64)
    for _, _, _, _, _, cls in dataset.items:
        cls_counts[cls] += 1
    cls_counts = np.clip(cls_counts, 1, None)
    weights = 1.0 / cls_counts
    sample_weights = [weights[cls] for *_, cls in dataset.items]
    return WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

def train_classifier_efficientnet_b0(img_size=224, epochs=20, batch_size=128, lr=1e-3, weight_decay=1e-3, seed=42, out_dir='runs_cls/effb0_224'):
    torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    # Use only the detector-train split images for crop training to avoid leakage
    train_img_names = set(p.name for p in (base / 'images' / 'train').glob('*.jpg'))
    ds = KuzuCropTrainDataset(train_csv, pool_train, img_names=train_img_names, img_size=img_size)
    sampler = create_sampler(ds)
    dl = DataLoader(ds, batch_size=batch_size, sampler=sampler, num_workers=8, pin_memory=True, drop_last=True)
    model = timm.create_model('tf_efficientnet_b0', pretrained=True, num_classes=num_classes)
    model = model.cuda()
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    best_loss = 1e9
    for ep in range(1, epochs+1):
        model.train()
        t0 = time.time()
        loss_sum = 0.0; n = 0
        for i, (imgs, targets) in enumerate(dl):
            imgs = imgs.cuda(non_blocking=True)
            targets = targets.cuda(non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(True):
                logits = model(imgs)
                loss = criterion(logits, targets)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            loss_sum += loss.item() * imgs.size(0); n += imgs.size(0)
            if (i+1) % 50 == 0:
                print(f'Epoch {ep} iter {i+1}: loss {loss.item():.4f}', flush=True)
        sched.step()
        avg_loss = loss_sum / max(1, n)
        print(f'Epoch {ep}/{epochs} avg_loss={avg_loss:.4f} time={time.time()-t0:.1f}s', flush=True)
        # Save checkpoint
        torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg_loss, 'uc2id': uc2id}, out / 'last.pth')
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg_loss, 'uc2id': uc2id}, out / 'best.pth')
    print('Classifier training complete. Best loss:', best_loss)

print('Classifier scaffolding ready. After detector threshold tuning, call train_classifier_efficientnet_b0() to train crops on GPU (not concurrently with YOLO).')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unicode classes: 4113
Classifier scaffolding ready. After detector threshold tuning, call train_classifier_efficientnet_b0() to train crops on GPU (not concurrently with YOLO).


In [29]:
# End-to-end inference helpers: classify detector boxes, build submission
import json, math, time
from pathlib import Path
import numpy as np
from PIL import Image
import torch
import timm
import torch.nn.functional as F

base = Path('kuzu_data')
pool_train = base / 'pool' / 'train'
test_img_dir = base / 'images' / 'test'

# Load Unicode maps
uc2id_path = base / 'unicode_to_id.json'
id2uc_path = base / 'id_to_unicode.json'
uc2id = json.loads(uc2id_path.read_text()) if uc2id_path.exists() else {}
if id2uc_path.exists():
    _id2uc_raw = json.loads(id2uc_path.read_text())
    id2uc = {int(k): v for k, v in _id2uc_raw.items()}
else:
    id2uc = {}

def load_classifier(weights_path, num_classes=None, device='cuda'):
    device = device if torch.cuda.is_available() else 'cpu'
    if num_classes is None:
        num_classes = len(id2uc) if id2uc else None
    assert num_classes is not None, 'num_classes must be known to init classifier'
    model = timm.create_model('tf_efficientnet_b0', pretrained=False, num_classes=num_classes)
    ckpt = torch.load(weights_path, map_location='cpu')
    missing = model.load_state_dict(ckpt['model'], strict=False)
    model.to(device).eval()
    return model, device

def crop_with_padding(im, x, y, w, h, pad_pct=0.04):
    # pad relative to box size (critical fix), then square-pad and return grayscale crop
    W, H = im.size
    pad = int(round(pad_pct * max(int(w), int(h))))
    x1 = max(0, int(x) - pad); y1 = max(0, int(y) - pad)
    x2 = min(W, int(x) + int(w) + pad); y2 = min(H, int(y) + int(h) + pad)
    if x2 <= x1: x2 = min(W, x1 + 1)
    if y2 <= y1: y2 = min(H, y1 + 1)
    crop = im.crop((x1, y1, x2, y2)).convert('L')
    cw, ch = crop.size; s = max(cw, ch)
    bg = Image.new('L', (s, s), color=255)
    bg.paste(crop, ((s - cw)//2, (s - ch)//2))
    return bg

def to_tensor_3ch(img_pil, size=224):
    # simple resize -> tensor [1,3,H,W] in 0..1
    im = img_pil.resize((size, size), Image.BILINEAR)
    arr = np.array(im, dtype=np.float32) / 255.0
    arr3 = np.stack([arr, arr, arr], axis=0)  # 3xHxW
    return torch.from_numpy(arr3)

def classify_boxes_on_images(cls_weights, dets_dict_path, img_dir, pad_pct=0.04, img_size=224, batch_size=256, conf_min=0.0):
    # dets_dict_path: .npy saved by detect_test or val preds; contains {img_name: [[x,y,w,h,conf], ...]}
    detections = np.load(dets_dict_path, allow_pickle=True).item()
    model, device = load_classifier(cls_weights, num_classes=len(id2uc))
    out_codes = {}  # img_name -> list of unicode strings
    with torch.no_grad():
        for img_name, boxes in detections.items():
            bxs = [b for b in boxes if b[4] >= conf_min]
            if not bxs:
                out_codes[img_name] = []
                continue
            img_path = Path(img_dir) / img_name
            with Image.open(img_path) as im:
                crops = [crop_with_padding(im, int(x), int(y), int(w), int(h), pad_pct=pad_pct) for (x,y,w,h,_) in bxs]
            # batch inference
            all_logits = []
            for i in range(0, len(crops), batch_size):
                batch = crops[i:i+batch_size]
                tens = torch.stack([to_tensor_3ch(c, size=img_size) for c in batch]).to(device)  # Bx3xHxW
                logits = model(tens)
                all_logits.append(logits.cpu())
            logits = torch.cat(all_logits, dim=0)
            pred_ids = logits.argmax(dim=1).tolist()
            codes = [id2uc.get(int(i), 'U+003F') for i in pred_ids]
            out_codes[img_name] = codes
    return out_codes

def write_submission(submit_path, dets_dict_path, codes_dict=None):
    # codes_dict: optional mapping img_name -> list of unicode codes aligned to dets; if None, uses 'U+003F'
    dets = np.load(dets_dict_path, allow_pickle=True).item()
    ss = Path('sample_submission.csv')
    import pandas as pd
    df_ss = pd.read_csv(ss)
    rows = []
    for _, r in df_ss.iterrows():
        img_id = r['image_id']
        img_name = f'{img_id}.jpg' if not img_id.lower().endswith('.jpg') else img_id
        boxes = dets.get(img_name, [])
        if not boxes:
            rows.append({'image_id': img_id, 'labels': ''})
            continue
        if codes_dict is None:
            codes = ['U+003F'] * len(boxes)  # placeholder
        else:
            codes = codes_dict.get(img_name, ['U+003F'] * len(boxes))
            if len(codes) != len(boxes):
                m = min(len(codes), len(boxes))
                codes = codes[:m]; boxes = boxes[:m]
        parts = []
        for (code, (x,y,w,h,_)) in zip(codes, boxes):
            parts.extend([code, str(int(x)), str(int(y)), str(int(max(1, int(w)))), str(int(max(1, int(h))))])
        rows.append({'image_id': img_id, 'labels': ' '.join(parts)})
    sub = pd.DataFrame(rows, columns=['image_id','labels'])
    sub.to_csv(submit_path, index=False)
    print('Wrote submission to', submit_path, 'shape', sub.shape)

print('End-to-end helpers ready: classify_boxes_on_images() and write_submission().')

End-to-end helpers ready: classify_boxes_on_images() and write_submission().


In [9]:
# Launch stronger YOLOv8m detector @1024 (class-agnostic) per expert plan
import time
from ultralytics import YOLO

t0 = time.time()
model = YOLO('yolov8m.pt')
print('Starting YOLOv8m training...', flush=True)
results = model.train(
    data='kuzu_data/kuzu.yaml',
    imgsz=1024,
    batch=12,
    epochs=80,
    device=0,
    amp=True,
    cos_lr=True,
    patience=20,
    warmup_epochs=3,
    rect=True,
    cache='ram',
    workers=8,
    close_mosaic=10,
    # Augs
    degrees=5.0,
    fliplr=0.0,
    flipud=0.0,
    translate=0.1,
    scale=0.3,
    shear=0.0,
    perspective=0.0001,
    hsv_h=0.015,
    hsv_s=0.45,
    hsv_v=0.3,
    mosaic=1.0,
    mixup=0.0,
    project='runs_det',
    name='yolov8m_1024_ca',
    pretrained=True,
    verbose=True,
    seed=42
)
print('YOLOv8m training launched; elapsed %.1f s' % (time.time()-t0), flush=True)
print('Save dir:', results.save_dir)

  0%|          | 0.00/49.7M [00:00<?, ?B/s]

  3%|▎         | 1.38M/49.7M [00:00<00:12, 3.97MB/s]

  8%|▊         | 4.00M/49.7M [00:00<00:04, 10.6MB/s]

 14%|█▍        | 7.12M/49.7M [00:00<00:02, 16.8MB/s]

 20%|██        | 10.1M/49.7M [00:00<00:02, 17.6MB/s]

...

KeyboardInterrupt: 

Exception in thread Thread-79 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner


    self.run()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.11/threading.py", line 975, in run


    self._target(*self._args, **self._kwargs)
  File "/app/.pip-target/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop


    do_one_step()
  File "/app/.pip-target/torch/utils/data/_utils/pin_memory.py", line 32, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/multiprocessing/reductions.py", line 496, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/resource_sharer.py", line 86, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 512, in Cli

In [10]:
# Stage-1 detector tuning (val) and train-split noisy crop mining (to run after GPU free)
import time
from pathlib import Path
import numpy as np
import pandas as pd

VAL_DIR = Path('kuzu_data/images/val')
TRAIN_DIR = Path('kuzu_data/images/train')

def yolov8_best_from(run_dir: str):
    rd = Path(run_dir)
    for p in [rd / 'weights' / 'best.pt', rd / 'weights' / 'last.pt']:
        if p.exists():
            return str(p)
    return None

def grid_search_stage1_det_only(run_dir='runs_det/yolov8s_1024_ca', imgsz=1024):
    from ultralytics import YOLO
    from scipy.optimize import linear_sum_assignment  # ensure available
    # Reuse helpers from cell 7 via globals: run_detector_on_val, eval_detector_only_f1
    w = yolov8_best_from(run_dir)
    assert w is not None, f'No weights found under {run_dir}'
    print('Stage-1 using weights:', w, flush=True)
    best = {'f1': -1, 'conf': None, 'iou': None, 'prec': None, 'rec': None}
    confs = [0.10, 0.15, 0.20, 0.25, 0.30]
    ious = [0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70]
    for conf in confs:
        for iou in ious:
            preds = run_detector_on_val(w, conf=float(conf), iou=float(iou), max_det=3000, agnostic_nms=True)
            f1, prec, rec, cnts = eval_detector_only_f1(preds, iou_thresh=0.5)
            print(f'[VAL] conf={conf:.2f} iou={iou:.2f} -> F1={f1:.4f} P={prec:.4f} R={rec:.4f} TP,FP,FN={cnts}', flush=True)
            if f1 > best['f1']:
                best.update({'f1': f1, 'conf': float(conf), 'iou': float(iou), 'prec': prec, 'rec': rec})
    print('Stage-1 best:', best, flush=True)
    return best

def detect_split_to_npy(weights_path, img_dir, out_path, imgsz=1024, conf=0.2, iou=0.55, max_det=3000):
    from ultralytics import YOLO
    img_dir = Path(img_dir)
    model = YOLO(weights_path)
    t0 = time.time()
    results = model.predict(source=str(img_dir), imgsz=imgsz, conf=conf, iou=iou,
                            agnostic_nms=True, max_det=max_det, verbose=False, save=False, stream=True, device=0)
    out = {}  # image_name -> list of [x,y,w,h,conf]
    n = 0
    for r in results:
        name = Path(r.path).name
        boxes = []
        if r.boxes is not None and len(r.boxes) > 0:
            xyxy = r.boxes.xyxy.cpu().numpy()
            confs = r.boxes.conf.cpu().numpy()
            for (x1,y1,x2,y2), c in zip(xyxy, confs):
                x = int(round(x1)); y = int(round(y1));
                w = max(1, int(round(x2 - x1))); h = max(1, int(round(y2 - y1)));
                boxes.append([x,y,w,h,float(c)])
        out[name] = boxes
        n += 1
        if n % 200 == 0:
            print(f'Processed {n} images in {time.time()-t0:.1f}s', flush=True)
    np.save(out_path, out, allow_pickle=True)
    print('Saved detections to', out_path, 'images:', len(out), 'elapsed:', f'{time.time()-t0:.1f}s')
    return out_path

def stage1_and_mine_with_yolov8s():
    # 1) Stage-1 grid on val (detector-only, yolov8s baseline)
    best = grid_search_stage1_det_only('runs_det/yolov8s_1024_ca', imgsz=1024)
    # 2) Mine noisy crops from train split using those thresholds
    w = yolov8_best_from('runs_det/yolov8s_1024_ca')
    assert w is not None, 'yolov8s weights not found'
    out_path = 'det_train_raw.npy'
    detect_split_to_npy(w, TRAIN_DIR, out_path, imgsz=1024, conf=best['conf'], iou=best['iou'], max_det=3000)
    print('Stage-1 complete. Detections saved at', out_path, 'Best:', best)
    return best, out_path

print('Prepared Stage-1 grid search and train-split mining utilities. Do NOT run while YOLOv8m is training.')

Prepared Stage-1 grid search and train-split mining utilities. Do NOT run while YOLOv8m is training.


In [22]:
# ConvNeXt-Tiny classifier on noisy+GT crops (prep; run after mining and when GPU free)
import os, json, math, time, random
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm

base = Path('kuzu_data')
pool_train = base / 'pool' / 'train'
train_df_full = pd.read_csv('train.csv')
uc2id = json.loads((base / 'unicode_to_id.json').read_text())
id2uc = {int(k): v for k, v in json.loads((base / 'id_to_unicode.json').read_text()).items()}
num_classes = len(uc2id)

def parse_label_string(s):
    if pd.isna(s) or not str(s).strip():
        return []
    tokens = str(s).strip().split()
    out = []; i = 0
    while i < len(tokens):
        t = tokens[i]
        if t.startswith('U+') and i + 5 <= len(tokens):
            try:
                x = int(tokens[i+1]); y = int(tokens[i+2]); w = int(tokens[i+3]); h = int(tokens[i+4])
                out.append((t, x, y, w, h))
            except Exception:
                pass
            i += 5
        else:
            i += 1
    return out

def make_img_lookup(img_names):
    return {Path(n).stem: n for n in img_names}

class NoisyCropsDataset(Dataset):
    # Mix of detector-mined crops (train split) and 20-30% GT crops
    def __init__(self, det_npy_path, mix_gt_ratio=0.25, img_size=224, pad_pct=0.06, jitter_pct=0.10, seed=42):
        random.seed(seed); np.random.seed(seed)
        self.img_size = img_size
        self.pad_pct = pad_pct
        self.jitter_pct = jitter_pct
        # train split image names only (to avoid leakage)
        train_imgs = sorted([p.name for p in (base / 'images' / 'train').glob('*.jpg')])
        self.lookup = make_img_lookup(train_imgs)
        self.img_root = pool_train
        # detector boxes
        det = np.load(det_npy_path, allow_pickle=True).item()
        det_items = []  # (img_name, x,y,w,h, cls_id)
        # Build GT dict per image for class labels
        gt_map = {}
        for _, r in train_df_full.iterrows():
            img_id = r['image_id']
            if img_id in self.lookup:
                gt_map[img_id + '.jpg'] = parse_label_string(r['labels'])
        # Assign class to det boxes by best IoU to GT in same image
        def iou(a, b):
            ax, ay, aw, ah = a; bx, by, bw, bh = b
            ax2, ay2 = ax+aw, ay+ah; bx2, by2 = bx+bw, by+bh
            iw = max(0, min(ax2, bx2) - max(ax, bx))
            ih = max(0, min(ay2, by2) - max(ay, by))
            inter = iw*ih
            if inter <= 0: return 0.0
            ua = aw*ah + bw*bh - inter
            return inter/ua if ua>0 else 0.0
        for img_name, boxes in det.items():
            if img_name not in self.lookup.values():
                continue
            gt_list = gt_map.get(img_name, [])
            for (x,y,w,h,conf) in boxes:
                best_iou, best_cls = 0.0, None
                for (code, gx, gy, gw, gh) in gt_list:
                    ii = iou((x,y,w,h), (gx,gy,gw,gh))
                    if ii > best_iou:
                        best_iou = ii
                        best_cls = uc2id.get(code, None)
                if best_cls is not None:
                    det_items.append((img_name, x, y, w, h, best_cls))
        # sample GT subset
        gt_items = []
        for img_name in self.lookup.values():
            img_id = Path(img_name).stem
            anns_df = train_df_full.loc[train_df_full['image_id']==img_id]
            if len(anns_df)==0: continue
            anns = parse_label_string(anns_df.iloc[0]['labels'])
            for (code,x,y,w,h) in anns:
                cls = uc2id.get(code, None)
                if cls is not None:
                    gt_items.append((img_name, x, y, w, h, cls))
        n_gt = int(mix_gt_ratio * len(det_items))
        gt_sel = random.sample(gt_items, k=min(n_gt, len(gt_items))) if gt_items else []
        self.items = det_items + gt_sel
        random.shuffle(self.items)
        # Augs: CLAHE, mild rotate/shear, brightness/contrast, noise, CoarseDropout, Normalize, ToTensorV2
        self.augs = A.Compose([
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=1.0),
            A.Resize(self.img_size, self.img_size, interpolation=1),
            A.Rotate(limit=5, border_mode=0, p=0.5),
            A.Affine(shear=(-2,2), p=0.4),
            A.RandomBrightnessContrast(0.15, 0.15, p=0.5),
            A.GaussNoise(var_limit=(5.0, 15.0), p=0.2),
            A.CoarseDropout(max_holes=2, max_height=int(0.10*self.img_size), max_width=int(0.10*self.img_size), p=0.3),
            A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
            ToTensorV2(),
        ])
        print('NoisyCropsDataset: det items', len(det_items), 'gt mix', len(gt_sel), 'total', len(self.items))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        img_name, x, y, w, h, cls = self.items[idx]
        with Image.open(self.img_root / img_name) as im:
            im = im.convert('L')
            W, H = im.size
            # jitter box
            j = self.jitter_pct
            dx = int(round((random.uniform(-j, j)) * w))
            dy = int(round((random.uniform(-j, j)) * h))
            dw = int(round((random.uniform(-j, j)) * w))
            dh = int(round((random.uniform(-j, j)) * h))
            xx, yy = max(0, x+dx), max(0, y+dy)
            ww, hh = max(1, w+dw), max(1, h+dh)
            pad = int(round(self.pad_pct * max(ww, hh)))
            x1 = max(0, xx - pad); y1 = max(0, yy - pad)
            x2 = min(W, xx + ww + pad); y2 = min(H, yy + hh + pad)
            if x2 <= x1: x2 = min(W, x1+1)
            if y2 <= y1: y2 = min(H, y1+1)
            crop = im.crop((x1, y1, x2, y2))
            # square pad
            cw, ch = crop.size; s = max(cw, ch)
            bg = Image.new('L', (s, s), color=255); bg.paste(crop, ((s-cw)//2, (s-ch)//2))
            arr = np.array(bg)
            arr = np.stack([arr, arr, arr], axis=-1)  # HxWx3 uint8
        out = self.augs(image=arr)['image']  # tensor
        if isinstance(out, torch.Tensor) and out.dtype != torch.float32:
            out = out.float()
        return out, torch.tensor(cls, dtype=torch.long)

def create_balanced_sampler(items, num_classes, samples_per_epoch=None):
    cls_counts = np.zeros(num_classes, dtype=np.int64)
    for _, _, _, _, _, cls in items:
        cls_counts[cls] += 1
    cls_counts = np.clip(cls_counts, 1, None)
    inv = 1.0 / cls_counts
    weights = [inv[cls] for *_, cls in items]
    if samples_per_epoch is None:
        samples_per_epoch = len(weights)
    return WeightedRandomSampler(weights, num_samples=int(samples_per_epoch), replacement=True)

def train_convnext_tiny(det_train_npy='det_train_raw.npy', img_size=224, epochs=25, batch_size=256, lr=5e-4, wd=1e-3, mix_gt_ratio=0.25, seed=42, out_dir='runs_cls/convnext_tiny_224', samples_per_epoch=80000):
    torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    ds = NoisyCropsDataset(det_train_npy, mix_gt_ratio=mix_gt_ratio, img_size=img_size, pad_pct=0.06, jitter_pct=0.10, seed=seed)
    sampler = create_balanced_sampler(ds.items, num_classes, samples_per_epoch=samples_per_epoch)
    dl = DataLoader(ds, batch_size=batch_size, sampler=sampler, num_workers=8, pin_memory=True, drop_last=True)
    model = timm.create_model('convnext_tiny.fb_in22k_ft_in1k', pretrained=True, num_classes=num_classes)
    model = model.cuda()
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scaler = torch.cuda.amp.GradScaler(enabled=False)  # disable AMP to avoid dtype mismatch
    best_loss = 1e9
    no_improve = 0
    for ep in range(1, epochs+1):
        t0 = time.time()
        model.train()
        run_loss, seen = 0.0, 0
        for i, (imgs, targets) in enumerate(dl):
            if isinstance(imgs, torch.Tensor) and imgs.dtype != torch.float32:
                imgs = imgs.float()
            imgs = imgs.cuda(non_blocking=True); targets = targets.cuda(non_blocking=True)
            opt.zero_grad(set_to_none=True)
            # no autocast to avoid uint8/half mismatch
            logits = model(imgs)
            loss = criterion(logits, targets)
            loss.backward()
            opt.step()
            run_loss += loss.item() * imgs.size(0); seen += imgs.size(0)
            if (i+1) % 50 == 0:
                print(f'[ep {ep}] iter {i+1} loss {loss.item():.4f}', flush=True)
        sched.step()
        avg = run_loss / max(1, seen)
        print(f'Epoch {ep}/{epochs} avg_loss={avg:.4f} time={time.time()-t0:.1f}s', flush=True)
        torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg, 'uc2id': uc2id}, out / 'last.pth')
        if avg < best_loss - 0.01:
            best_loss = avg; no_improve = 0
            torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg, 'uc2id': uc2id}, out / 'best.pth')
        else:
            no_improve += 1
        if ep >= 8 and no_improve >= 3:
            print('Early stopping triggered.'); break
    print('ConvNeXt-T training done. Best loss:', best_loss)

print('ConvNeXt-T classifier prep ready. Workflow: run stage1_and_mine_with_yolov8s() -> train_convnext_tiny(). Do not run during YOLO training.')

ConvNeXt-T classifier prep ready. Workflow: run stage1_and_mine_with_yolov8s() -> train_convnext_tiny(). Do not run during YOLO training.


In [30]:
# End-to-end (require_class=True) validation and joint threshold tuning (prepare; run when GPU free)
import time, json
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from PIL import Image

# Reuse id2uc from cell 10 when that cell is executed
base = Path('kuzu_data')
val_img_dir = base / 'images' / 'val'

# Box IoU and matching (class-aware)
def iou_xywh(a, b):
    ax, ay, aw, ah = a; bx, by, bw, bh = b
    ax2, ay2 = ax+aw, ay+ah; bx2, by2 = bx+bw, by+bh
    iw = max(0, min(ax2, bx2) - max(ax, bx))
    ih = max(0, min(ay2, by2) - max(ay, by))
    inter = iw*ih
    if inter <= 0: return 0.0
    ua = aw*ah + bw*bh - inter
    return inter/ua if ua>0 else 0.0

def hungarian_match(gt_boxes, pr_boxes, iou_thresh=0.5, require_class=True):
    from scipy.optimize import linear_sum_assignment
    if len(gt_boxes)==0 and len(pr_boxes)==0: return 0,0,0,[]
    if len(gt_boxes)==0: return 0,len(pr_boxes),0,[]
    if len(pr_boxes)==0: return 0,0,len(gt_boxes),[]
    G, P = len(gt_boxes), len(pr_boxes)
    cost = np.ones((G,P), dtype=np.float32)
    for i,(gcode,gx,gy,gw,gh) in enumerate(gt_boxes):
        for j,(pcode,px,py,pw,ph,pc) in enumerate(pr_boxes):
            if require_class and (pcode is None or pcode!=gcode):
                continue
            iou = iou_xywh((gx,gy,gw,gh),(px,py,pw,ph))
            if iou >= iou_thresh:
                cost[i,j] = 1.0 - iou
    gi, pj = linear_sum_assignment(cost)
    tp, used = 0, set()
    for a,b in zip(gi,pj):
        if cost[a,b] < 1.0:
            tp += 1; used.add(b)
    fp = P - len(used); fn = G - tp
    return tp, fp, fn, list(zip(gi,pj))

def f1_from_counts(tp, fp, fn):
    prec = tp/(tp+fp) if (tp+fp)>0 else 0.0
    rec = tp/(tp+fn) if (tp+fn)>0 else 0.0
    return (2*prec*rec/(prec+rec) if (prec+rec)>0 else 0.0), prec, rec

# GT loader for val
def load_val_gt():
    train_csv = pd.read_csv('train.csv')
    names = sorted([p.name for p in val_img_dir.glob('*.jpg')])
    def parse_label_string(s):
        if pd.isna(s) or not str(s).strip(): return []
        t = str(s).strip().split(); out=[]; i=0
        while i < len(t):
            if t[i].startswith('U+') and i+5<=len(t):
                try:
                    code=t[i]; x=int(t[i+1]); y=int(t[i+2]); w=int(t[i+3]); h=int(t[i+4]); out.append((code,x,y,w,h))
                except: pass
                i+=5
            else:
                i+=1
        return out
    gt = {}
    for n in names:
        img_id = Path(n).stem
        r = train_csv.loc[train_csv['image_id']==img_id]
        gt[n] = parse_label_string(r.iloc[0]['labels']) if len(r)==1 else []
    return gt

# Detector inference on val to get raw boxes
def detect_val_to_dict(weights_path, imgsz=1024, conf=0.2, iou=0.55, max_det=3000):
    from ultralytics import YOLO
    model = YOLO(weights_path)
    results = model.predict(source=str(val_img_dir), imgsz=imgsz, conf=conf, iou=iou,
                            agnostic_nms=True, max_det=max_det, verbose=False, save=False, stream=True, device=0)
    out = {}
    for r in results:
        name = Path(r.path).name; boxes=[]
        if r.boxes is not None and len(r.boxes)>0:
            xyxy = r.boxes.xyxy.cpu().numpy(); confs = r.boxes.conf.cpu().numpy()
            for (x1,y1,x2,y2),c in zip(xyxy, confs):
                x=int(round(x1)); y=int(round(y1)); w=max(1,int(round(x2-x1))); h=max(1,int(round(y2-y1)))
                boxes.append([x,y,w,h,float(c)])
        out[name]=boxes
    return out

# Classify detections into Unicode codes AND top-1 probabilities
def classify_detections_for_dir(cls_weights_path, dets_dict, img_dir, pad_pct=0.06, img_size=224, batch_size=256, device='cuda'):
    # Requires crop_with_padding and to_tensor_3ch from cell 10 to be executed
    import timm, torch
    from torch.nn import functional as F
    model = timm.create_model('convnext_tiny.fb_in22k_ft_in1k', pretrained=False, num_classes=len(id2uc))
    ckpt = torch.load(cls_weights_path, map_location='cpu')
    model.load_state_dict(ckpt['model'], strict=False)
    device = device if torch.cuda.is_available() else 'cpu'
    model.to(device).eval()
    out = {}  # name -> list of (unicode, pmax)
    with torch.no_grad():
        for name, boxes in dets_dict.items():
            if not boxes:
                out[name]=[]; continue
            with Image.open(Path(img_dir)/name) as im:
                crops = [crop_with_padding(im, int(x),int(y),int(w),int(h), pad_pct=pad_pct) for (x,y,w,h,_) in boxes]
            logits_all = []
            for i in range(0, len(crops), batch_size):
                batch = crops[i:i+batch_size]
                tens = torch.stack([to_tensor_3ch(c, size=img_size) for c in batch]).to(device)
                logits = model(tens)
                logits_all.append(logits.cpu())
            logits = torch.cat(logits_all, dim=0)
            probs = torch.softmax(logits, dim=1)
            pmax, pred_ids = probs.max(dim=1)
            codes = [id2uc.get(int(i), 'U+003F') for i in pred_ids.tolist()]
            out[name] = [(codes[i], float(pmax[i])) for i in range(len(codes))]
    return out

# Assemble predictions with classes for F1 evaluation
def build_pr_list_with_codes(dets_dict, codes_probs_dict, score_mode='product'):
    # score_mode: 'product' uses det_conf * cls_pmax and stores score in last field
    pr = {}
    for name, boxes in dets_dict.items():
        items = []
        cp = codes_probs_dict.get(name, [])
        m = min(len(boxes), len(cp))
        for i in range(m):
            code, pmax = cp[i]
            x,y,w,h,dc = boxes[i]
            score = float(dc) * float(pmax) if score_mode=='product' else float(dc)
            items.append((code, int(x), int(y), int(w), int(h), score))
        pr[name] = items
    return pr

def eval_end_to_end_f1(dets_with_codes, val_gt, iou_thresh=0.5):
    tp=fp=fn=0
    for name, gt in val_gt.items():
        pr = dets_with_codes.get(name, [])
        tpp,fpp,fnn,_ = hungarian_match(gt, pr, iou_thresh=iou_thresh, require_class=True)
        tp+=tpp; fp+=fpp; fn+=fnn
    f1, prec, rec = f1_from_counts(tp, fp, fn)
    return f1, prec, rec, (tp,fp,fn)

# Joint tuning using combined score s = det_conf * cls_prob
def joint_search_end_to_end(det_weights, cls_weights, iou_fixed=0.55,
                            conf_list=(0.14,0.16,0.18,0.20,0.22,0.24,0.26),
                            s_list=(0.10,0.12,0.14,0.16,0.18,0.20),
                            pad_pct=0.06, cls_img_size=224):
    val_gt = load_val_gt()
    best = {'f1':-1}
    for conf in conf_list:
        dets = detect_val_to_dict(det_weights, imgsz=1024, conf=float(conf), iou=float(iou_fixed), max_det=3000)
        cp = classify_detections_for_dir(cls_weights, dets, val_img_dir, pad_pct=pad_pct, img_size=cls_img_size, batch_size=256)
        pr_full = build_pr_list_with_codes(dets, cp, score_mode='product')
        # Offline threshold sweep on product score
        for s in s_list:
            pr = {}
            for name, items in pr_full.items():
                pr[name] = [it for it in items if float(it[5]) >= float(s)]
            f1,prec,rec,cnts = eval_end_to_end_f1(pr, val_gt, iou_thresh=0.5)
            print(f'[VAL end2end] conf={conf:.3f} iou={iou_fixed:.2f} s={s:.3f} -> F1={f1:.4f} P={prec:.4f} R={rec:.4f} {cnts}', flush=True)
            if f1 > best.get('f1',-1):
                best = {'f1':f1,'prec':prec,'rec':rec,'tp_fp_fn':cnts,'conf':float(conf),'iou':float(iou_fixed),'s':float(s),'pad_pct':pad_pct,'cls_img_size':cls_img_size}
    print('Best end-to-end setting:', best, flush=True)
    return best

print('End-to-end joint tuning scaffold ready. After training classifier and detector grid Stage-1, execute joint_search_end_to_end() to pick final thresholds.')

End-to-end joint tuning scaffold ready. After training classifier and detector grid Stage-1, execute joint_search_end_to_end() to pick final thresholds.


In [13]:
# Run Stage-1 grid search and mine train-split detections using yolov8s
import time
t0 = time.time()
best, det_train_path = stage1_and_mine_with_yolov8s()
print('Stage-1 completed in %.1fs' % (time.time()-t0))
print('Best detector-only settings:', best)
print('Detections saved to:', det_train_path)

Stage-1 using weights: runs_det/yolov8s_1024_ca/weights/best.pt


[VAL] conf=0.10 iou=0.40 -> F1=0.9630 P=0.9428 R=0.9841 TP,FP,FN=(58099, 3523, 936)


[VAL] conf=0.10 iou=0.45 -> F1=0.9606 P=0.9374 R=0.9850 TP,FP,FN=(58150, 3883, 885)


[VAL] conf=0.10 iou=0.50 -> F1=0.9579 P=0.9315 R=0.9857 TP,FP,FN=(58191, 4277, 844)


[VAL] conf=0.10 iou=0.55 -> F1=0.9540 P=0.9238 R=0.9863 TP,FP,FN=(58224, 4803, 811)


...

Processed 2200 images in 63.8s


Processed 2400 images in 68.9s


Processed 2600 images in 73.6s


Processed 2800 images in 78.2s


Saved detections to det_train_raw.npy images: 2919 elapsed: 81.0s
Stage-1 complete. Detections saved at det_train_raw.npy Best: {'f1': 0.9736144426208811, 'conf': 0.3, 'iou': 0.4, 'prec': 0.9675476748076279, 'rec': 0.9797577708139239}
Stage-1 completed in 666.4s
Best detector-only settings: {'f1': 0.9736144426208811, 'conf': 0.3, 'iou': 0.4, 'prec': 0.9675476748076279, 'rec': 0.9797577708139239}
Detections saved to: det_train_raw.npy


In [34]:
# Inline ConvNeXt-Tiny training with bf16 AMP, ImageNet norm, IoU-filtered pseudo-labels
from pathlib import Path
import time, json, random, numpy as np, pandas as pd, os
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision('medium')

base = Path('kuzu_data')
pool_train = base / 'pool' / 'train'
train_df_full = pd.read_csv('train.csv')
uc2id = json.loads((base / 'unicode_to_id.json').read_text())
id2uc = {int(k): v for k, v in json.loads((base / 'id_to_unicode.json').read_text()).items()}
num_classes = len(uc2id)

def parse_label_string(s):
    if pd.isna(s) or not str(s).strip():
        return []
    t = str(s).strip().split(); out=[]; i=0
    while i < len(t):
        if t[i].startswith('U+') and i+5<=len(t):
            try:
                code=t[i]; x=int(t[i+1]); y=int(t[i+2]); w=int(t[i+3]); h=int(t[i+4]); out.append((code,x,y,w,h))
            except: pass
            i+=5
        else:
            i+=1
    return out

def make_img_lookup(img_names):
    return {Path(n).stem: n for n in img_names}

class NoisyCropsDataset(Dataset):
    def __init__(self, det_npy_path, mix_gt_ratio=0.50, img_size=224, pad_pct=0.06, jitter_pct=0.10, seed=42, iou_min=0.30, det_conf_min=0.20):
        random.seed(seed); np.random.seed(seed)
        self.img_size = img_size; self.pad_pct = pad_pct; self.jitter_pct = jitter_pct
        train_imgs = sorted([p.name for p in (base / 'images' / 'train').glob('*.jpg')])
        self.lookup = make_img_lookup(train_imgs)
        self.img_root = pool_train
        det = np.load(det_npy_path, allow_pickle=True).item()
        det_items = []; gt_map = {}
        for _, r in train_df_full.iterrows():
            img_id = r['image_id']
            if img_id in self.lookup:
                gt_map[img_id + '.jpg'] = parse_label_string(r['labels'])
        def iou(a,b):
            ax,ay,aw,ah=a; bx,by,bw,bh=b
            ax2,ay2=ax+aw,ay+ah; bx2,by2=bx+bw,by+bh
            iw=max(0,min(ax2,bx2)-max(ax,bx)); ih=max(0,min(ay2,by2)-max(ay,by)); inter=iw*ih
            if inter<=0: return 0.0
            ua=aw*ah+bw*bh-inter; return inter/ua if ua>0 else 0.0
        for img_name, boxes in det.items():
            if img_name not in self.lookup.values():
                continue
            gt_list = gt_map.get(img_name, [])
            for (x,y,w,h,conf) in boxes:
                if float(conf) < det_conf_min:
                    continue
                best_i, best_cls = 0.0, None
                for (code,gx,gy,gw,gh) in gt_list:
                    ii = iou((x,y,w,h),(gx,gy,gw,gh))
                    if ii > best_i: best_i, best_cls = ii, uc2id.get(code, None)
                if best_cls is not None and best_i >= iou_min:
                    det_items.append((img_name,x,y,w,h,best_cls))
        gt_items = []
        for img_name in self.lookup.values():
            img_id = Path(img_name).stem
            r = train_df_full.loc[train_df_full['image_id']==img_id]
            if len(r)==0: continue
            for (code,x,y,w,h) in parse_label_string(r.iloc[0]['labels']):
                cls = uc2id.get(code, None)
                if cls is not None: gt_items.append((img_name,x,y,w,h,cls))
        n_gt = int(mix_gt_ratio * len(det_items))
        gt_sel = random.sample(gt_items, k=min(n_gt, len(gt_items))) if gt_items else []
        self.items = det_items + gt_sel
        random.shuffle(self.items)
        self.augs = A.Compose([
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=1.0),
            A.Resize(self.img_size, self.img_size, interpolation=1),
            A.Rotate(limit=5, border_mode=0, p=0.5),
            A.Affine(shear=(-2,2), p=0.4),
            A.RandomBrightnessContrast(0.15,0.15,p=0.5),
            A.GaussNoise(var_limit=(5.0,15.0), p=0.2),
            A.CoarseDropout(max_holes=2, max_height=int(0.10*self.img_size), max_width=int(0.10*self.img_size), p=0.3),
            # ImageNet normalization for timm pretrained
            A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
            ToTensorV2(),
        ])
        print('NoisyCropsDataset: det items', len(det_items), 'gt mix', len(gt_sel), 'total', len(self.items))
    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        img_name,x,y,w,h,cls = self.items[idx]
        with Image.open(self.img_root / img_name) as im:
            im = im.convert('L'); W,H = im.size
            j=self.jitter_pct
            dx=int(round((random.uniform(-j,j))*w)); dy=int(round((random.uniform(-j,j))*h))
            dw=int(round((random.uniform(-j,j))*w)); dh=int(round((random.uniform(-j,j))*h))
            xx,yy=max(0,x+dx),max(0,y+dy); ww,hh=max(1,w+dw),max(1,h+dh)
            pad=int(round(self.pad_pct * max(ww,hh)))
            x1=max(0,xx-pad); y1=max(0,yy-pad); x2=min(W,xx+ww+pad); y2=min(H,yy+hh+pad)
            if x2<=x1: x2=min(W,x1+1)
            if y2<=y1: y2=min(H,y1+1)
            crop = im.crop((x1,y1,x2,y2))
            cw,ch = crop.size; s=max(cw,ch)
            bg = Image.new('L',(s,s),color=255); bg.paste(crop,((s-cw)//2,(s-ch)//2))
            arr = np.array(bg); arr = np.stack([arr,arr,arr], axis=-1)
        out = self.augs(image=arr)['image']
        if isinstance(out, torch.Tensor) and out.dtype != torch.float32: out = out.float()
        return out, torch.tensor(cls, dtype=torch.long)

def create_balanced_sampler(items, num_classes, samples_per_epoch=40000):
    cls_counts = np.zeros(num_classes, dtype=np.int64)
    for *_, cls in items: cls_counts[cls]+=1
    cls_counts=np.clip(cls_counts,1,None); inv=1.0/cls_counts
    weights=[inv[cls] for *_,cls in items]
    return WeightedRandomSampler(weights, num_samples=int(samples_per_epoch), replacement=True)

# Train inline
det_npy = Path('det_train_raw.npy')
assert det_npy.exists(), 'det_train_raw.npy not found yet. Run Stage-1 (cell 15) first.'
print('Found', det_npy, '-> starting ConvNeXt-Tiny training (inline)...')
t0 = time.time()
epochs=15; batch_size=64; lr=1e-3; wd=1e-3; img_size=224; seed=42; out_dir=Path('runs_cls/convnext_tiny_224')
torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)
out_dir.mkdir(parents=True, exist_ok=True)
ds = NoisyCropsDataset(str(det_npy), mix_gt_ratio=0.50, img_size=img_size, pad_pct=0.06, jitter_pct=0.10, seed=seed, iou_min=0.30, det_conf_min=0.20)
sampler = create_balanced_sampler(ds.items, num_classes, samples_per_epoch=40000)
dl = DataLoader(ds, batch_size=batch_size, sampler=sampler, num_workers=4, pin_memory=True, drop_last=True)
model = timm.create_model('convnext_tiny.fb_in22k_ft_in1k', pretrained=True, num_classes=num_classes).cuda()
model = model.to(memory_format=torch.channels_last)
# Disable grad checkpointing for bring-up
if hasattr(model, 'set_grad_checkpointing'):
    try:
        model.set_grad_checkpointing(False)
    except Exception as e:
        print('Grad checkpointing disable failed:', e)
opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
best_loss = 1e9; no_improve=0
warmup_steps = 800  # ~ first 800 iters linear warmup to lr
global_step = 0
for ep in range(1, epochs+1):
    ep_t0=time.time(); model.train(); run_loss=0.0; seen=0
    for i,(imgs,targets) in enumerate(dl):
        if isinstance(imgs, torch.Tensor) and imgs.dtype != torch.float32: imgs = imgs.float()
        imgs = imgs.to(memory_format=torch.channels_last)
        imgs=imgs.cuda(non_blocking=True); targets=targets.cuda(non_blocking=True)
        # Warmup LR
        if global_step < warmup_steps:
            warmup_lr = lr * (global_step+1) / warmup_steps
            for pg in opt.param_groups: pg['lr'] = warmup_lr
        opt.zero_grad(set_to_none=True)
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
            logits = model(imgs)
            loss = criterion(logits, targets)
        loss.backward(); opt.step()
        global_step += 1
        run_loss += loss.item()*imgs.size(0); seen += imgs.size(0)
        if (i+1)%50==0:
            torch.cuda.synchronize()
            print(f'[ep {ep}] iter {i+1} loss {loss.item():.4f}', flush=True)
    # restore base lr after warmup
    for pg in opt.param_groups: pg['lr'] = lr
    sched.step()
    avg = run_loss/max(1,seen)
    print(f'Epoch {ep}/{epochs} avg_loss={avg:.4f} time={time.time()-ep_t0:.1f}s', flush=True)
    torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg, 'uc2id': uc2id}, out_dir/'last.pth')
    if avg < best_loss - 0.01:
        best_loss = avg; no_improve=0; torch.save({'model': model.state_dict(), 'epoch': ep, 'loss': avg, 'uc2id': uc2id}, out_dir/'best.pth')
    else:
        no_improve += 1
    if ep>=8 and no_improve>=3:
        print('Early stopping triggered.'); break
print('ConvNeXt-Tiny training finished in %.1f min' % ((time.time()-t0)/60.0))
print('Best loss:', best_loss)

Found det_train_raw.npy -> starting ConvNeXt-Tiny training (inline)...


NoisyCropsDataset: det items 551177 gt mix 275588 total 826765


[ep 1] iter 50 loss 8.3970


[ep 1] iter 100 loss 8.3266


[ep 1] iter 150 loss 8.3397


...

[ep 7] iter 50 loss 0.9511


[ep 7] iter 100 loss 0.9060


[ep 7] iter 150 loss 0.9539


[ep 7] iter 200 loss 0.9570


KeyboardInterrupt: 

In [15]:
# Fix albumentations dependency: install compatible albucore
import sys, subprocess, pkgutil
print('Installing albucore>=0.0.13 for albumentations>=1.4.x...', flush=True)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', 'albucore==0.0.13', '--upgrade', '--upgrade-strategy', 'only-if-needed'], check=True)
import albucore, albumentations as A
print('albucore version:', getattr(albucore, '__version__', 'unknown'))
print('albumentations version:', getattr(A, '__version__', 'unknown'))
print('Albumentations import OK')

Installing albucore>=0.0.13 for albumentations>=1.4.x...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 394.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 223.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 219.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 288.1 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.5 requires opencv-python>=4.6.0, which is not installed.


ImportError: cannot import name 'preserve_channel_dim' from 'albucore.utils' (/app/.pip-target/albucore/utils.py)

In [16]:
# Fix Albumentations import by downgrading to a stable version without albucore
import sys, subprocess
def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Remove conflicting versions
pip('uninstall', '-y', 'albumentations', 'albucore')

# Install Albumentations 1.3.1 (no albucore dependency)
pip('install', '-c', 'constraints.txt', 'albumentations==1.3.1', '--upgrade-strategy', 'only-if-needed')

import albumentations as A
from albumentations.pytorch import ToTensorV2
print('Albumentations OK:', getattr(A, '__version__', 'unknown'))

> pip uninstall -y albumentations albucore


Found existing installation: albumentations 1.4.14
Uninstalling albumentations-1.4.14:
  Successfully uninstalled albumentations-1.4.14
Found existing installation: albucore 0.0.33
Uninstalling albucore-0.0.33:
  Successfully uninstalled albucore-0.0.33
> pip install -c constraints.txt albumentations==1.3.1 --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 KB 137.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 234.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 222.5 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 318.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 KB 506.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 202.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 418.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 KB 497.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.5 requires opencv-python>=4.6.0, which is not installed.


Albumentations OK: 1.3.1


In [37]:
# Submission helper: product-gated classification and writer
import json, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import torch

base = Path('kuzu_data')
test_img_dir = base / 'images' / 'test'

def make_submission_with_product(det_npy_path, cls_weights_path, out_csv='submission.csv',
                                  s_thresh=0.16, pad_pct=0.06, cls_img_size=224,
                                  drop_min_w=6, drop_min_h=6):
    # Uses classify_detections_for_dir() from cell 14 and crop_with_padding/to_tensor_3ch from cell 10
    dets = np.load(det_npy_path, allow_pickle=True).item()  # name -> [[x,y,w,h,det_conf], ...]
    # Classify all detections once
    cp = classify_detections_for_dir(cls_weights_path, dets, test_img_dir, pad_pct=pad_pct, img_size=cls_img_size, batch_size=256)
    # Build submission rows in sample_submission order
    ss = pd.read_csv('sample_submission.csv')
    rows = []
    for _, r in ss.iterrows():
        img_id = r['image_id']
        name = f'{img_id}.jpg' if not img_id.lower().endswith('.jpg') else img_id
        boxes = dets.get(name, [])
        codes_probs = cp.get(name, [])
        parts = []
        if boxes and codes_probs:
            m = min(len(boxes), len(codes_probs))
            # score = det_conf * cls_prob, filter and sort by (y,x) for stability
            items = []
            for i in range(m):
                code, pmax = codes_probs[i]
                x,y,w,h,dc = boxes[i]
                if w < drop_min_w or h < drop_min_h:
                    continue
                score = float(dc) * float(pmax)
                if score >= float(s_thresh):
                    items.append((y, x, code, int(x), int(y), int(max(1,int(w))), int(max(1,int(h)))))
            items.sort(key=lambda z: (z[0], z[1]))
            for _, _, code, x, y, w, h in items:
                parts.extend([code, str(x), str(y), str(w), str(h)])
        rows.append({'image_id': img_id, 'labels': ' '.join(parts)})
    sub = pd.DataFrame(rows, columns=['image_id','labels'])
    sub.to_csv(out_csv, index=False)
    print('Wrote submission:', out_csv, 'shape:', sub.shape)
    return out_csv

print('Submission helper ready: call make_submission_with_product(det_test_raw.npy, cls_best.pth, s_thresh, pad_pct).')

Submission helper ready: call make_submission_with_product(det_test_raw.npy, cls_best.pth, s_thresh, pad_pct).


In [35]:
# Run joint end-to-end tuning with product score (widened grids)
best = joint_search_end_to_end(
    det_weights='runs_det/yolov8s_1024_ca/weights/best.pt',
    cls_weights='runs_cls/convnext_tiny_224/best.pth',
    iou_fixed=0.55,
    conf_list=[0.24,0.26,0.28,0.30,0.32],
    s_list=[0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16],
    pad_pct=0.06,
    cls_img_size=224,
)
print('Joint tuning best:', best)

[VAL end2end] conf=0.240 iou=0.55 s=0.020 -> F1=0.6748 P=0.6655 R=0.6844 (40403, 20305, 18632)


[VAL end2end] conf=0.240 iou=0.55 s=0.040 -> F1=0.6775 P=0.6709 R=0.6841 (40388, 19811, 18647)


[VAL end2end] conf=0.240 iou=0.55 s=0.060 -> F1=0.6821 P=0.6819 R=0.6823 (40281, 18787, 18754)


[VAL end2end] conf=0.240 iou=0.55 s=0.080 -> F1=0.6879 P=0.6966 R=0.6794 (40108, 17467, 18927)


[VAL end2end] conf=0.240 iou=0.55 s=0.100 -> F1=0.6927 P=0.7110 R=0.6753 (39867, 16205, 19168)


...

[VAL end2end] conf=0.320 iou=0.55 s=0.120 -> F1=0.6963 P=0.7259 R=0.6691 (39499, 14917, 19536)


[VAL end2end] conf=0.320 iou=0.55 s=0.140 -> F1=0.6991 P=0.7391 R=0.6633 (39158, 13824, 19877)


[VAL end2end] conf=0.320 iou=0.55 s=0.160 -> F1=0.7007 P=0.7511 R=0.6566 (38765, 12847, 20270)


Best end-to-end setting: {'f1': 0.7007983527201792, 'prec': 0.7505658406360629, 'rec': 0.6572202930464979, 'tp_fp_fn': (38799, 12894, 20236), 'conf': 0.28, 'iou': 0.55, 's': 0.16, 'pad_pct': 0.06, 'cls_img_size': 224}


Joint tuning best: {'f1': 0.7007983527201792, 'prec': 0.7505658406360629, 'rec': 0.6572202930464979, 'tp_fp_fn': (38799, 12894, 20236), 'conf': 0.28, 'iou': 0.55, 's': 0.16, 'pad_pct': 0.06, 'cls_img_size': 224}


In [38]:
# Prepare test detections and build submission
print('Extracting and linking test images...', flush=True)
extract_test_images()
link_test_images()

print('Running test detection with tuned thresholds...', flush=True)
det_path = detect_test(
    weights_path='runs_det/yolov8s_1024_ca/weights/best.pt',
    out_path='det_test_raw.npy',
    conf=0.28,
    iou=0.55,
    max_det=3000
)
print('Detections saved to:', det_path, flush=True)

print('Classifying detections and writing submission...', flush=True)
sub_path = make_submission_with_product(
    det_npy_path=det_path,
    cls_weights_path='runs_cls/convnext_tiny_224/best.pth',
    out_csv='submission.csv',
    s_thresh=0.16,
    pad_pct=0.06,
    cls_img_size=224,
    drop_min_w=2,
    drop_min_h=2
)
print('Submission ready at', sub_path)
import os; print('submission.csv size:', os.path.getsize('submission.csv'))

Extracting and linking test images...


Extracted 100/361 in 0.2s


Extracted 200/361 in 0.5s


Extracted 300/361 in 0.7s


Test extraction complete: 361
Linked test images: 361
Running test detection with tuned thresholds...


Saved detections to det_test_raw.npy with 361 images
Detections saved to: det_test_raw.npy


Classifying detections and writing submission...


Wrote submission: submission.csv shape: (361, 2)
Submission ready at submission.csv
submission.csv size: 1401942


In [ ]:
# Rebuild submission in point format: labels as 'U+XXXX x y' (no w,h) to match sample_submission schema
import numpy as np, pandas as pd
from pathlib import Path
from PIL import Image

# Reuse classify_detections_for_dir from cell 14 and id2uc/crop utils from cell 10
det_path = 'det_test_raw.npy'
cls_path = 'runs_cls/convnext_tiny_224/best.pth'
s_thresh = 0.16
pad_pct = 0.06
img_size = 224
drop_min_w = 2
drop_min_h = 2

dets = np.load(det_path, allow_pickle=True).item()  # name -> [[x,y,w,h,dc],...]
print('Loaded dets for', len(dets), 'images')
cp = classify_detections_for_dir(cls_path, dets, Path('kuzu_data/images/test'), pad_pct=pad_pct, img_size=img_size, batch_size=256)
print('Classified images:', len(cp))

ss = pd.read_csv('sample_submission.csv')
rows = []
for _, r in ss.iterrows():
    img_id = r['image_id']
    name = f'{img_id}.jpg' if not img_id.lower().endswith('.jpg') else img_id
    boxes = dets.get(name, [])
    codes_probs = cp.get(name, [])
    parts = []
    if boxes and codes_probs:
        m = min(len(boxes), len(codes_probs))
        items = []
        for i in range(m):
            code, pmax = codes_probs[i]
            x,y,w,h,dc = boxes[i]
            if w < drop_min_w or h < drop_min_h:
                continue
            score = float(dc) * float(pmax)
            if score >= s_thresh:
                cx = int(x + w/2)
                cy = int(y + h/2)
                items.append((y, x, code, cx, cy))
        items.sort(key=lambda z: (z[0], z[1]))
        for _, _, code, cx, cy in items:
            parts.extend([code, str(cx), str(cy)])
    rows.append({'image_id': img_id, 'labels': ' '.join(parts)})
sub = pd.DataFrame(rows, columns=['image_id','labels'])
sub.to_csv('submission.csv', index=False)
print('Rewrote submission.csv in point format, shape:', sub.shape)

Loaded dets for 361 images
